# Importing libraries

In [10]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline,Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
data = pd.read_csv('spam_classification.csv', encoding='latin-1')

# EDA

In [18]:
data = data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4']).rename(columns={'v1':'target','v2':'msg'})
data

,target,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# Creating and training the model

In [19]:
# train_df, test_df = train_test_split(data, test_size=0.20, random_state=42)

# X_train, y_train = train_df["msg"], train_df["target"]
# X_test, y_test = test_df["msg"], test_df["target"]

# clf = make_pipeline( CountVectorizer(max_features=5000),LogisticRegression(max_iter=5000))
# clf.fit(X_train,y_train)
x=data['msg']
y=data['target']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=123)
cntv= make_pipeline(CountVectorizer(max_features=5000))
lgr= make_pipeline(LogisticRegression(max_iter=5000,random_state=123))
pipe_lgr = make_pipeline(cntv,lgr)
pipe_lgr.fit(x_train, y_train)

Pipeline(steps=[('pipeline-1',
                 Pipeline(steps=[('countvectorizer',
                                  CountVectorizer(max_features=5000))])),
                ('pipeline-2',
                 Pipeline(steps=[('logisticregression',
                                  LogisticRegression(max_iter=5000,
                                                     random_state=123))]))])

In [20]:
prediction = pipe_lgr.predict(x_test)
result = pd.DataFrame({'test_data':x_test,'original_label':y_test,'prediction':prediction})
result['original_label'][result['original_label']==result['prediction']].count()

1096

# score on LogisticRegression

In [21]:
score = pipe_lgr.score(x_test,y_test)
print(score)

0.9829596412556054


# Result dataframe

In [29]:
wrong_prediction = result[result['original_label']!=result['prediction']]
wrong_prediction

,test_data,original_label,prediction
671,SMS. ac sun0819 posts HELLO:\You seem cool,spam,ham
730,Email AlertFrom: Jeri StewartSize: 2KBSubject:...,spam,ham
1448,As a registered optin subscriber ur draw 4 å£1...,spam,ham
1268,Can U get 2 phone NOW? I wanna chat 2 set up m...,spam,ham
2769,Burger King - Wanna play footy at a top stadiu...,spam,ham
5370,dating:i have had two of these. Only started a...,spam,ham
3690,"Sir, i am waiting for your call, once free ple...",ham,spam
2246,Hi ya babe x u 4goten bout me?' scammers getti...,spam,ham
1673,Monthly password for wap. mobsi.com is 391784....,spam,ham
2401,Babe: U want me dont u baby! Im nasty and have...,spam,ham


In [27]:
result['original_label'].value_counts()

ham     982
spam    133
Name: original_label, dtype: int64

# confusion matrix

In [23]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions
predictions = pipe_lgr.predict(x_test)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, predictions))


Confusion Matrix:
[[981   1]
 [ 18 115]]

Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       982
        spam       0.99      0.86      0.92       133

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

